Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist im Beispiel ein Apache-Webserver installiert, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service apache --namespace ms-kubernetes -o=jsonpath='{ .spec.ports[0].nodePort }')/"   


Das Microservices Zusammenspiel verfolgen wird in der Weave Scope Oberfläche.

Mittels den nachfolgenden Befehlen wird der Port der Weave Scope Oberfläche gegen aussen (IP VM) freigeschaltet.

In [ ]:
! kubectl get service weave-scope-app -n weave -o yaml | sed 's/ClusterIP/NodePort/' | kubectl apply -f -
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service weave-scope-app --namespace weave -o=jsonpath='{ .spec.ports[0].nodePort }')/")'/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["ms-kubernetes"]}}}'

***
### REST Paradigma

Representational State Transfer (abgekürzt REST, seltener auch ReST) bezeichnet ein Programmierparadigma für verteilte Systeme.

REST ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web (HTTP GET, PUT, POST, DELETE).

REST hat das Ziel, einen Architekturstil zu schaffen, der die Anforderungen des modernen Web besser darstellt. Dabei unterscheidet sich REST vor allem in der Forderung nach einer einheitlichen Kommunikation Schnittstelle von anderen Architekturstilen.

Um das REST Paradigma zu verdeutlichen, führen wir mittels `curl` (Windows `Invoke-WebRequest`) mehrere API Aufrufe gegen unsere Microservices aus:

In [ ]:
! curl http://apache.ms-kubernetes/customer/customer

In [ ]:
! curl http://apache.ms-kubernetes/catalog/catalog

In [ ]:
! curl http://apache.ms-kubernetes/order/order/<Ersetzen mit order id>

***
### ReplicaSet

Um die ReplicaSet Funktionalität zu Demonstrieren, setzen wir die Anzahl der Pods `customer, order, catalog` auf 3 und schauen uns das Ergebnis an.

In [ ]:
! kubectl --namespace rs scale --replicas=3 deployment/customer --namespace ms-kubernetes
! kubectl --namespace rs scale --replicas=3 deployment/order --namespace ms-kubernetes
! kubectl --namespace rs scale --replicas=3 deployment/catalog --namespace ms-kubernetes
! kubectl get pods --namespace ms-kubernetes

***
### Deployments

Das Menu vom `apache` Web Server zeigt immer noch das Hystrix Dashboard an. Dieses wird in einem Kubernetes Umfeld nicht mehr benötigt. Deshalb ersetzen wir das Container Image durch die neuste Version.


In [ ]:
! kubectl set image deployment/apache apache=ewolff/microservice-kubernetes-demo-apache:latest --namespace ms-kubernetes

In [ ]:
! kubectl describe deployment/apache --namespace ms-kubernetes

- - -

Aufräumen

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/hystrix.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml